In [1]:
import ibapi
import threading

from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract


In [2]:
class IBWrapper(EWrapper):
    def __init__(self):
        EWrapper.__init__(self)
        
class IBClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

In [3]:

class App(IBClient, IBWrapper):

    def __init__(self):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)

In [4]:
#app = App()
# 
#app.connect("127.0.0.1", 4002, 14)

In [1]:
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=1)
print(ib.isConnected())




True


In [71]:
ib.disconnect()
print(ib.isConnected())

False


In [68]:
import numpy as np
import pandas as pd
from math import atan2, pi
import datetime
from datetime import timedelta

from statistics import stdev

contract = Stock('AAPL', 'SMART', 'USD', primaryExchange='NASDAQ')

from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, intercept, rvalue, _, _ = linregress(x, returns)
    momo = ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2
    momo_price = np.e ** (intercept + slope*x)
    return  momo


def atr(df, n=20):
    data = df.copy()
    high = data.high
    low = data.low
    close = data.close
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = tr.ewm(alpha=1/n, min_periods=n, adjust=False).mean() #J. Welles Wilder's EMA 
    return tr, atr

def angle(df, period):
    return np.rad2deg(np.arctan2(df['average']-df['average'].shift(period-1), period-1))
    
    
def addon_5m_tone1(df):
    df = df.copy()
    df['5m_1st_high'] = 0.000
    df['5m_1st_low'] = 0.000
    df['5m_1st_range'] = 0.000
    df['5m_1st_volume'] = 0
    
    df['5m_2nd_high'] = 0.000
    df['5m_2nd_low'] = 0.000
    df['5m_2nd_range'] = 0.000
    df['5m_2nd_volume'] = 0
    
    
    for i, row in df.iterrows():
        fixed_date = str(row.date).split(" ")[0]
        datetime = fixed_date.replace('-', '') + ' ' + '20:00:00'
        print(fixed_date)
        print(datetime) #20191127 20:00:00
        df5m = reqHistData(contract, endDateTime=datetime, durationStr='1 D', barSizeSetting='5 mins')
        print(df5m.index.date[0]) #2019-11-26 04:20:00
        print('--------')
        if fixed_date == str(df5m.index.date[0]):
            df5m_1st = df5m.loc[(df5m.index.hour == 9) & (df5m.index.minute == 30)].iloc[0]
            df5m_2nd = df5m.loc[(df5m.index.hour == 9) & (df5m.index.minute == 35)].iloc[0]
            df.at[i, '5m_1st_high'] = df5m_1st.high
            df.at[i, '5m_1st_low'] = df5m_1st.low
            df.at[i, '5m_1st_range'] = df5m_1st.high-df5m_1st.low
            df.at[i, '5m_1st_volume'] = df5m_1st.volume
            
            df.at[i, '5m_2nd_high'] = df5m_2nd.high
            df.at[i, '5m_2nd_low'] = df5m_2nd.low
            df.at[i, '5m_2nd_range'] = df5m_2nd.high - df5m_2nd.low
            df.at[i, '5m_2nd_volume'] = df5m_2nd.volume
    
    return df


def addon_5m_tone(df, batch='1 M'):
    df = df.copy()
    df['5m_1st_high'] = 0.000
    df['5m_1st_low'] = 0.000
    df['5m_1st_range'] = 0.000
    df['5m_1st_volume'] = 0
    df['5m_1st_drvol'] = 0.0
    df['5m_1st_drvol'] = df['5m_1st_drvol'].astype(int)
    
    df['5m_2nd_high'] = 0.000
    df['5m_2nd_low'] = 0.000
    df['5m_2nd_range'] = 0.000
    df['5m_2nd_volume'] = 0
    df['5m_2nd_drvol'] = 0.0
    df['5m_2nd_drvol'] = df['5m_2nd_drvol'].astype(int)
    
    df['stdev50_volume'] = stdev(df['volume'], 50)
    
    print('daily start -->' + str(df[:1].index.date[0])) #2018-12-21
    print('daily end -->' + str(df[-1:].index.date[0]))
    
    start = str(df[:1].index.date[0]).split(" ")[0].replace('-', '')
    end = str(df[-1:].index.date[0]).split(" ")[0].replace('-', '')
    
    fixed_end = end + ' ' + '20:00:00'
    
    
    while True:
        print('batching -------->')
        print(fixed_end)
        dfm = reqHistData(contract, endDateTime=fixed_end, durationStr=batch, barSizeSetting='5 mins')

        print('5m start -->' + str(dfm[:1].index.date[0])) #2019-11-21
        print('5m end -->' + str(dfm[-1:].index.date[0]))
        start_5m = dfm[:1].index.date[0]
        end_5m = dfm[-1:].index.date[0]
        
        if ( 
            start_5m <= df[-1:].index.date[0] and 
            end_5m <= df[-1:].index.date[0] and
            end_5m > df[:1].index.date[0]
        ):
# batching -------->
# 20190121 20:00:00
# 5m start -->2018-12-20
# 5m end -->2019-01-18
# new 5m start -->2018-12-19
# daily start -->2018-12-21

            if start_5m < df[:1].index.date[0]:
                start_5m = df[:1].index.date[0]
        
        
            for i, row in df.loc[start_5m:end_5m].iterrows():
                row5_30 = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
                               & (dfm.index.hour == 9) & (dfm.index.minute == 30)]
                row5_35 = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
                               & (dfm.index.hour == 9) & (dfm.index.minute == 35)]
                
                if row5_30.empty or row5_35.empty:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!skipped:" + str(i))
                    continue
                
                dfm_1st = row5_30.iloc[0] 
                dfm_2nd = row5_35.iloc[0]
                
                df.at[i, '5m_1st_high'] = dfm_1st.high
                df.at[i, '5m_1st_low'] = dfm_1st.low
                df.at[i, '5m_1st_range'] = dfm_1st.close - dfm_1st.open
                df.at[i, '5m_1st_volume'] = dfm_1st.volume

                df.at[i, '5m_2nd_high'] = dfm_2nd.high
                df.at[i, '5m_2nd_low'] = dfm_2nd.low
                df.at[i, '5m_2nd_range'] = dfm_2nd.close - dfm_2nd.open
                df.at[i, '5m_2nd_volume'] = dfm_2nd.volume
                
                if not (pd.isna(row.avg50_volume) or pd.isna(row.stdev50_volume or pd.isna(dfm_1st.volume) or pd.isna(dfm_2nd.volume))):
                    #print(i)
                    #print(dfm_1st.volume)
                    #print(row.avg50_volume)
                    #print(row.stdev50_volume)
                    df.at[i, '5m_1st_drvol'] = (dfm_1st.volume - row.avg50_volume) / row.stdev50_volume
                    df.at[i, '5m_2nd_drvol'] = (dfm_2nd.volume - row.avg50_volume) / row.stdev50_volume
                
                p_o = datetime.time(4, 0, 0)
                p_c = datetime.time(9, 25, 0)
                dfm_premarket = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
                               & (dfm.index.time >= p_o) & (dfm.index.time <= p_c)]    
                df.at[i, 'premarket_high'] = dfm_premarket['high'].max()
                df.at[i, 'premarket_low'] = dfm_premarket['low'].min()
                
                a_o = datetime.time(16, 5, 0)
                a_c = datetime.time(20, 0, 0)
                dfm_premarket = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
                               & (dfm.index.time >= a_o) & (dfm.index.time <= a_c)]    
                df.at[i, 'aftermarket_high'] = dfm_premarket['high'].max()
                df.at[i, 'aftermarket_low'] = dfm_premarket['low'].min()
            
            
            
            
        print('new 5m start -->' + str(dfm[:1].index.date[0] - timedelta(days=1)))
        print('daily start -->' + str(df[:1].index.date[0]))
        
        if dfm[:1].index.date[0] - timedelta(days=1) < df[:1].index.date[0]:
            break
        
        new_end = str(dfm[:1].index.date[0] - timedelta(days=1)).split(" ")[0].replace('-', '') + ' ' + '20:00:00'
        fixed_end = new_end

        
#     dfm = reqHistData(contract, endDateTime=fixed_end, durationStr='1 M', barSizeSetting='5 mins')

#     print('5m start -->' + str(dfm[:1].index))
#     print('5m end -->' + str(dfm[-1:].index))

    
#     if (
#         start == dfm[:1].index.date[0].strftime("%Y%m%d") and 
#         end == dfm[-1:].index.date[0].strftime("%Y%m%d")
#     ):
#         for i, row in df.iterrows():
#             dfm_1st = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
#                            & (dfm.index.hour == 9) & (dfm.index.minute == 30)].iloc[0]
#             dfm_2nd = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
#                            & (dfm.index.hour == 9) & (dfm.index.minute == 35)].iloc[0]

#             df.at[i, '5m_1st_high'] = dfm_1st.high
#             df.at[i, '5m_1st_low'] = dfm_1st.low
#             df.at[i, '5m_1st_range'] = dfm_1st.high - dfm_1st.low
#             df.at[i, '5m_1st_volume'] = dfm_1st.volume
            
#             df.at[i, '5m_2nd_high'] = dfm_2nd.high
#             df.at[i, '5m_2nd_low'] = dfm_2nd.low
#             df.at[i, '5m_2nd_range'] = dfm_2nd.high - dfm_2nd.low
#             df.at[i, '5m_2nd_volume'] = dfm_2nd.volume
    
    return df

def addon_30m(df, batch='3 M'):
    df = df.copy()
    df['30m_1st_high'] = 0.000
    df['30m_1st_low'] = 0.000
    df['30m_1st_range'] = 0.000
    df['30m_1st_volume'] = 0
    
    start = str(df[:1].index.date[0]).split(" ")[0].replace('-', '')
    end = str(df[-1:].index.date[0]).split(" ")[0].replace('-', '')
    
    fixed_end = end + ' ' + '20:00:00'
    
    while True:
        print('batching -------->')
        print(fixed_end)
        dfm = reqHistData(contract, endDateTime=fixed_end, durationStr=batch, barSizeSetting='30 mins')

        print('30m start -->' + str(dfm[:1].index.date[0])) #2019-11-21
        print('30m end -->' + str(dfm[-1:].index.date[0]))
        start_30m = dfm[:1].index.date[0]
        end_30m = dfm[-1:].index.date[0]
        
        if ( 
            start_30m <= df[-1:].index.date[0] and 
            end_30m <= df[-1:].index.date[0] and
            end_30m > df[:1].index.date[0]
        ):

            if start_30m < df[:1].index.date[0]:
                start_30m = df[:1].index.date[0]
        
            for i, row in df.loc[start_30m:end_30m].iterrows():
                
                row30_30 = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
                               & (dfm.index.hour == 9) & (dfm.index.minute == 30)]
                
                if row30_30.empty:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!skipped:" + str(i))
                    continue
                    
                #print(i.to_pydatetime().date())
                dfm_1st = row30_30.iloc[0]

                df.at[i, '30m_1st_high'] = dfm_1st.high
                df.at[i, '30m_1st_low'] = dfm_1st.low
                df.at[i, '30m_1st_range'] = dfm_1st.close - dfm_1st.open
                df.at[i, '30m_1st_volume'] = dfm_1st.volume
        
        print('new 30m start -->' + str(dfm[:1].index.date[0] - timedelta(days=1)))
        print('daily start -->' + str(df[:1].index.date[0]))
        
        if dfm[:1].index.date[0] - timedelta(days=1) < df[:1].index.date[0]:
            break
        
        new_end = str(dfm[:1].index.date[0] - timedelta(days=1)).split(" ")[0].replace('-', '') + ' ' + '20:00:00'
        fixed_end = new_end
    
    return df

def addon_10m(df, batch='3 M'):
    df = df.copy()
    df['10m_50ema'] = 0.000
    df['10m_200ema'] = 0.000
    
    start = str(df[:1].index.date[0]).split(" ")[0].replace('-', '')
    end = str(df[-1:].index.date[0]).split(" ")[0].replace('-', '')
    
    fixed_end = end + ' ' + '20:00:00'
    
    while True:
        print('batching -------->')
        print(fixed_end)
        dfm = reqHistData(contract, endDateTime=fixed_end, durationStr=batch, barSizeSetting='10 mins')

        print('10m start -->' + str(dfm[:1].index.date[0])) #2019-11-21
        print('10m end -->' + str(dfm[-1:].index.date[0]))
        start_10m = dfm[:1].index.date[0]
        end_10m = dfm[-1:].index.date[0]
        
        if ( 
            start_10m <= df[-1:].index.date[0] and 
            end_10m <= df[-1:].index.date[0] and
            end_10m > df[:1].index.date[0]
        ):

            if start_10m < df[:1].index.date[0]:
                start_10m = df[:1].index.date[0]
        
            for i, row in df.loc[start_10m:end_10m].iterrows():
                
                row10_30 = dfm.loc[(dfm.year == i.year) & (dfm.month == i.month) & (dfm.day == i.day) \
                               & (dfm.index.hour == 9) & (dfm.index.minute == 30)]
                
                if row10_30.empty:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!skipped:" + str(i))
                    continue
                
                #print(i.to_pydatetime().date())
                dfm_1st = row10_30.iloc[0]

                df.at[i, '10m_50ema'] = dfm_1st['50ema']
                df.at[i, '10m_200ema'] = dfm_1st['200ema']
        
        print('new 10m start -->' + str(dfm[:1].index.date[0] - timedelta(days=1)))
        print('daily start -->' + str(df[:1].index.date[0]))
        
        if dfm[:1].index.date[0] - timedelta(days=1) < df[:1].index.date[0]:
            break
        
        new_end = str(dfm[:1].index.date[0] - timedelta(days=1)).split(" ")[0].replace('-', '') + ' ' + '20:00:00'
        fixed_end = new_end
    
    return df
    
    
# The formula for the calculation of the exponential moving average is recursively defined as follows:

# EMA1 = price1;

# EMA2 = α*price2 + (1 - α)*EMA1;

# EMA3 = α*price3 + (1 - α)*EMA2;

# EMAN = α*priceN + (1 - α)*EMAN-1;

# where α is a smoothing coefficient equal to 2/(length + 1).

# Note that in thinkScript®, exponential moving averages use prefetch, see the Past Offset and Prefetch article for details.



def reqHistData(contract, endDateTime, durationStr, barSizeSetting):
    extend_hour = False
    
    if 'day' in barSizeSetting: #only use intraday data
        extend_hour = True
    
    bars = ib.reqHistoricalData(contract, endDateTime=endDateTime, durationStr=durationStr,
        barSizeSetting=barSizeSetting, whatToShow='TRADES', useRTH=extend_hour)
    df = util.df(bars)


#     df['ts'] = df.date.astype('int64') // 10**9

    if (
        'hour' in barSizeSetting or 
        'mins' in barSizeSetting or 
        'secs' in barSizeSetting
    ):
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df.date.dt.year
        df['month'] = df.date.dt.month
        df['day'] = df.date.dt.day
        df['hour'] = df.date.dt.hour
        df['minute'] = df.date.dt.minute
    else:
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df.date.dt.year
        df['month'] = df.date.dt.month
        df['day'] = df.date.dt.day
        
    df.reset_index()
    df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df =  df.sort_index(ascending=True)
    
    df['21ema'] = df['close'].ewm(span=21,adjust=True,ignore_na=False).mean() #min_periods=0,
    df['50ema'] = df['close'].ewm(span=50,adjust=True,ignore_na=False).mean()
    df['200ema'] = df['close'].ewm(span=200,adjust=True,ignore_na=False).mean()

    if (
        '3 secs' in barSizeSetting or
        '1 min' in barSizeSetting or
        '5 min' in barSizeSetting
    ):
        #vwap
        df['v*p'] = df['close'] * df['volume']
        df['total vp'] = df['v*p'].cumsum()
        df['total v'] = df['volume'].cumsum()
        df['vwap'] = df['total vp']/df['total v']

    if '1 day' in barSizeSetting:
        df['gap'] = 100 * (df['open'] - df['close'].shift(1)) / df['close'].shift(1)
        df['avg50_volume'] = df['volume'].rolling(50).mean()
        #df['avg50_volume'].fillna(0, inplace=True)
        df['2d_angle'] = angle(df, 2)
        #df['momo'], df['momo_price'] = zip(*df['close'].rolling(90).apply(momentum, raw=False))
        df['90d_momo'] = df['close'].rolling(90).apply(momentum, raw=False)
        df['tr'], df['atr20'] = atr(df, 20)
        #  multiday run, average range of homerunner, earnings(EPS, sales, forward guidance) with gap down > 5%,  first 1m rvol, highRelVol >= 4, acc_volume based on tick size
        df = addon_5m_tone(df)
        #df = addon_30m(df)
        #df = addon_10m(df)
        
#         input period = aggregationPeriod.DAY;
#         def rawRelVol = (volume(period=period) - Average(volume(period=period), 50)) / StDev(volume(period=period), 50);
#         def highRelVol = Max(0, rawRelVol);
    
    #print(type(df.index))
    #print(barSizeSetting)
    #print(df.iloc[(df.index.hour == 9) & (df.index.minute == 35)])
    #print(df.tail(10))
    #print("\n")
    return df


#dfd = reqHistData(contract, endDateTime='20191231 20:00:00', durationStr='1 Y', barSizeSetting='1 day')
#dfd.drop(columns=['barCount'], inplace =True)
# print(dfd.head(10))
#print(dfd.tail(10))
# print(dfd.isna().sum())
# print(dfd[dfd == 0].count(axis=0))
#df4h = reqHistData(contract, endDateTime='20191222 20:00:00', durationStr='1 W', barSizeSetting='4 hours')

#df1m = reqHistData(contract, endDateTime='20191223 20:00:00', durationStr='1 D', barSizeSetting='30 secs')

In [14]:
print(dfd.isna().sum())
print(dfd[dfd == 0].count(axis=0))

open                 0
high                 0
low                  0
close                0
volume               0
average              0
year                 0
month                0
day                  0
21ema                0
50ema                0
200ema               0
gap                  1
avg50_volume        49
2d_angle             1
90d_momo            89
tr                   0
atr20               19
5m_1st_high          0
5m_1st_low           0
5m_1st_range         0
5m_1st_volume        0
5m_1st_drvol         0
5m_2nd_high          0
5m_2nd_low           0
5m_2nd_range         0
5m_2nd_volume        0
5m_2nd_drvol         0
stdev50_volume       0
premarket_high       0
premarket_low        0
aftermarket_high     0
aftermarket_low      0
dtype: int64
open                 0
high                 0
low                  0
close                0
volume               0
average              0
year                 0
month                0
day                  0
21ema                

In [29]:
import os

# start = 2018
# substr = '1231 20:00:00'
# end = 2018
# durationStr='1 Y'

# while True:
#     endDateTime = str(start) + substr

#     df = reqHistData(contract, endDateTime=endDateTime, durationStr=durationStr, barSizeSetting='1 day')
    
#     file = str(contract.symbol) + '_' + str(start) + '.csv'
#     path = "E:\Day Trading\Historical Data"
#     output_file = os.path.join(path, file)
#     export_csv = df.to_csv(output_file, header=True)
    
#     start -= 1
#     if start < end:
#         print("done")
#         break
    
    
df = reqHistData(contract, endDateTime='20191231 20:00:00', durationStr='30 Y', barSizeSetting='1 day')
    
file = str(contract.symbol) + '_' + str(start) + '_30Y' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

daily start -->1990-01-08
daily end -->2019-12-31
batching -------->
20191231 20:00:00
5m start -->2019-12-02
5m end -->2019-12-31
new 5m start -->2019-12-01
daily start -->1990-01-08
batching -------->
20191201 20:00:00
5m start -->2019-10-31
5m end -->2019-11-29
new 5m start -->2019-10-30
daily start -->1990-01-08
batching -------->
20191030 20:00:00
5m start -->2019-10-01
5m end -->2019-10-30
new 5m start -->2019-09-30
daily start -->1990-01-08
batching -------->
20190930 20:00:00
5m start -->2019-09-03
5m end -->2019-09-30
new 5m start -->2019-09-02
daily start -->1990-01-08
batching -------->
20190902 20:00:00
5m start -->2019-08-01
5m end -->2019-08-30
new 5m start -->2019-07-31
daily start -->1990-01-08
batching -------->
20190731 20:00:00
5m start -->2019-07-02
5m end -->2019-07-31
new 5m start -->2019-07-01
daily start -->1990-01-08
batching -------->
20190701 20:00:00
5m start -->2019-06-03
5m end -->2019-07-01
new 5m start -->2019-06-02
daily start -->1990-01-08
batching ---

5m start -->2014-11-03
5m end -->2014-12-01
new 5m start -->2014-11-02
daily start -->1990-01-08
batching -------->
20141102 20:00:00
5m start -->2014-10-02
5m end -->2014-10-31
new 5m start -->2014-10-01
daily start -->1990-01-08
batching -------->
20141001 20:00:00
5m start -->2014-09-02
5m end -->2014-10-01
new 5m start -->2014-09-01
daily start -->1990-01-08
batching -------->
20140901 20:00:00
5m start -->2014-07-31
5m end -->2014-08-29
new 5m start -->2014-07-30
daily start -->1990-01-08
batching -------->
20140730 20:00:00
5m start -->2014-07-01
5m end -->2014-07-30
new 5m start -->2014-06-30
daily start -->1990-01-08
batching -------->
20140630 20:00:00
5m start -->2014-06-02
5m end -->2014-06-30
new 5m start -->2014-06-01
daily start -->1990-01-08
batching -------->
20140601 20:00:00
5m start -->2014-05-01
5m end -->2014-05-30
new 5m start -->2014-04-30
daily start -->1990-01-08
batching -------->
20140430 20:00:00
5m start -->2014-04-01
5m end -->2014-04-30
new 5m start -->20

new 5m start -->2009-09-30
daily start -->1990-01-08
batching -------->
20090930 20:00:00
5m start -->2009-09-01
5m end -->2009-09-30


IndexError: single positional indexer is out-of-bounds

In [69]:
df5 = reqHistData(contract, endDateTime='20090930 20:00:00', durationStr='1 M', barSizeSetting='5 mins')
print(df5.isna().sum())
print(df5[df5 == 0].count(axis=0))
print(df5.loc[(df5.index.hour == 9) & (df5.index.minute == 35) & (df5.index.day == 29)])
df5d = reqHistData(contract, endDateTime='20090930 20:00:00', durationStr='1 M', barSizeSetting='1 day')

open        0
high        0
low         0
close       0
volume      0
average     0
barCount    0
year        0
month       0
day         0
hour        0
minute      0
21ema       0
50ema       0
200ema      0
v*p         0
total vp    0
total v     0
vwap        0
dtype: int64
open          0
high          0
low           0
close         0
volume      524
average       0
barCount    524
year          0
month         0
day           0
hour          0
minute      286
21ema         0
50ema         0
200ema        0
v*p         524
total vp      0
total v       0
vwap          0
dtype: int64
                      open   high   low  close  volume    average  barCount  \
date                                                                          
2009-09-29 09:35:00  26.61  26.75  26.6  26.69   28980  26.665143      2397   

                     year  month  day  hour  minute      21ema      50ema  \
date                                                                        
2009-09-29 0

In [42]:
import pandas



# if (isPeriodRolled) {
#     volumeSum = volume;
#     volumeVwapSum = volume * vwap;
# } else {
#     volumeSum = compoundValue(1, volumeSum[1] + volume, volume);
#     volumeVwapSum = compoundValue(1, volumeVwapSum[1] + volume * vwap, volume * vwap);
# }
# def price = volumeVwapSum / volumeSum;


def vwap(df):
    volumes = df.volume
    prices = df.average
    highs = df.high
    lows = df.low
    return df.assign(vwap = (((prices * volumes).cumsum() / volumes.cumsum()))) #((highs + lows + prices) / 3)


#df['year'] = df.date.dt.year
#df['month'] = df.date.dt.month
#df['day'] = df.date.dt.day
#df['hour'] = df.date.dt.hour
#df['minute'] = df.date.dt.minute
#vwap = vwap(df)
#print(df.reset_index().groupby(by=['year', 'month', 'day']).count())
print(df1m.get(size=5))
print(df1m[(df1m['month'] == 12)&(df1m['day'] == 20)&(df1m['hour'] == 9)])

TypeError: get() got an unexpected keyword argument 'size'